# Section: Ball Throw

Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/ball_throw](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/ball_throw)

## This program demonstrates double deallocation of an array.

From ChatGPT 4o:

Let's break down the physical equations governing the motion of the ball, how they are discretized using the Runge-Kutta 4th order (RK4) method, and how the motion of the ball is simulated step-by-step in the code.

### **1. Physical Model (Equations of Motion)**

The ball is subject to gravity, and we are solving for its trajectory. This can be modeled by the following system of **ordinary differential equations (ODEs)**:

#### State Variables:
- **Position in x**: $ X(t) $
- **Position in y**: $ Y(t) $
- **Velocity in x**: $ V_X(t) $
- **Velocity in y**: $ V_Y(t) $

#### Equations of Motion:
The motion of the ball is governed by Newton’s laws of motion. The system of equations is:

1. **Velocity equations** (derivatives of position):
   $$
   \frac{dX}{dt} = V_X(t)
   $$
   $$
   \frac{dY}{dt} = V_Y(t)
   $$

2. **Acceleration equations** (derivatives of velocity):
   $$
   \frac{dV_X}{dt} = 0 \quad (\text{No horizontal acceleration due to air resistance})
   $$
   $$
   \frac{dV_Y}{dt} = -g = -9.81 \, \text{m/s}^2 \quad (\text{Acceleration due to gravity})
   $$

So the full system of ODEs can be written as:

$$
\frac{d}{dt} \begin{bmatrix} X \\ Y \\ V_X \\ V_Y \end{bmatrix} = \begin{bmatrix} V_X \\ V_Y \\ 0 \\ -g \end{bmatrix}
$$

Where:
- $ X(t) $ and $ Y(t) $ are the positions in the horizontal and vertical directions.
- $ V_X(t) $ and $ V_Y(t) $ are the velocities in the horizontal and vertical directions.
- $ g = 9.81 \, \text{m/s}^2 $ is the acceleration due to gravity.

### **2. Discretization of the Equations Using the Runge-Kutta Method**

The Runge-Kutta 4th order method (RK4) is a numerical method for solving ordinary differential equations. It approximates the solution by calculating intermediate values of the derivatives at multiple points within a time step and combining them to achieve a higher-order approximation.

The system of ODEs can be written as:

$$
\frac{d\mathbf{u}}{dt} = \mathbf{f}(t, \mathbf{u})
$$

Where $ \mathbf{u}(t) $ is the vector of state variables, $ \mathbf{u}(t) = \begin{bmatrix} X(t) \\ Y(t) \\ V_X(t) \\ V_Y(t) \end{bmatrix} $, and $ \mathbf{f}(t, \mathbf{u}) $ is the right-hand side (RHS) function, which is computed by the `rhs` subroutine.

In your program, the equations of motion are:

$$
\mathbf{f}(t, \mathbf{u}) = \begin{bmatrix} V_X \\ V_Y \\ 0 \\ -g \end{bmatrix}
$$

#### Step 1: The Runge-Kutta Method

The RK4 method computes the next state vector $ \mathbf{u}(t + \Delta t) $ by evaluating the derivatives at four points within the time step $ \Delta t $ and then combining them. Here's the mathematical formulation:

Let $ t_0 $ be the current time, and $ \mathbf{u}_0 $ be the current state vector. The Runge-Kutta 4th order method computes the next state $ \mathbf{u}_1 $ at $ t_0 + \Delta t $ as:

$$
k_1 = \Delta t \, \mathbf{f}(t_0, \mathbf{u}_0)
$$
$$
k_2 = \Delta t \, \mathbf{f}\left(t_0 + \frac{\Delta t}{2}, \mathbf{u}_0 + \frac{k_1}{2}\right)
$$
$$
k_3 = \Delta t \, \mathbf{f}\left(t_0 + \frac{\Delta t}{2}, \mathbf{u}_0 + \frac{k_2}{2}\right)
$$
$$
k_4 = \Delta t \, \mathbf{f}(t_0 + \Delta t, \mathbf{u}_0 + k_3)
$$

The new state vector $ \mathbf{u}_1 $ is given by a weighted sum of these intermediate values:

$$
\mathbf{u}_1 = \mathbf{u}_0 + \frac{1}{6} \left(k_1 + 2k_2 + 2k_3 + k_4\right)
$$

This formula combines the values at four points: the beginning of the step, the midpoint (evaluated twice), and the end of the step.

#### Step 2: Applying the RK4 Method to Your System

Now, let's apply the RK4 method to your system of equations.

The state vector is:
$$
\mathbf{u} = \begin{bmatrix} X \\ Y \\ V_X \\ V_Y \end{bmatrix}
$$

The RHS function is:
$$
\mathbf{f}(t, \mathbf{u}) = \begin{bmatrix} V_X \\ V_Y \\ 0 \\ -g \end{bmatrix}
$$

We need to calculate the four approximations ( $ k_1 $, $ k_2 $, $ k_3 $, and $ k_4 $ ) for each of the state variables:

1. **At the current state $ t_0 $, $ \mathbf{u}_0 $**:
   $$
   k_1 = \Delta t \, \begin{bmatrix} V_X \\ V_Y \\ 0 \\ -g \end{bmatrix}
   $$

2. **At the midpoint $ t_0 + \frac{\Delta t}{2} $, using $ k_1 $ to update $ \mathbf{u}_0 $**:
   $$
   k_2 = \Delta t \, \begin{bmatrix} V_X + \frac{k_1(3)}{2} \\ V_Y + \frac{k_1(4)}{2} \\ 0 \\ -g \end{bmatrix}
   $$

3. **At the midpoint again, $ t_0 + \frac{\Delta t}{2} $, using $ k_2 $ to update $ \mathbf{u}_0 $**:
   $$
   k_3 = \Delta t \, \begin{bmatrix} V_X + \frac{k_2(3)}{2} \\ V_Y + \frac{k_2(4)}{2} \\ 0 \\ -g \end{bmatrix}
   $$

4. **At the end of the step $ t_0 + \Delta t $, using $ k_3 $ to update $ \mathbf{u}_0 $**:
   $$
   k_4 = \Delta t \, \begin{bmatrix} V_X + k_3(3) \\ V_Y + k_3(4) \\ 0 \\ -g \end{bmatrix}
   $$

#### Final Update:
Finally, the updated state $ \mathbf{u}_1 $ is calculated as:

$$
\mathbf{u}_1 = \mathbf{u}_0 + \frac{1}{6} \left(k_1 + 2k_2 + 2k_3 + k_4\right)
$$

For each component (X, Y, $ V_X $, $ V_Y $), you apply this formula, updating the state vector by adding the weighted sum of the intermediate steps.

---

### **3. Discretization of the Motion**

#### X and Y (Position):

1. For position $ X $, the derivative is $ \frac{dX}{dt} = V_X $.
2. For position $ Y $, the derivative is $ \frac{dY}{dt} = V_Y $.

At each step, the velocities $ V_X $ and $ V_Y $ are updated, so the positions $ X $ and $ Y $ are updated based on the velocities.

#### $ V_X $ (Velocity in x-direction):
- The velocity in the x-direction $ V_X $ remains constant (no acceleration in the x-direction):
  $$
  \frac{dV_X}{dt} = 0
  $$
  This means that $ V_X $ doesn't change during the simulation.

#### $ V_Y $ (Velocity in y-direction):

- The velocity in the y-direction is affected by gravity:
  $$
  \frac{dV_Y}{dt} = -g
  $$
  The velocity $ V_Y $ decreases over time due to the downward acceleration caused by gravity.

---

### **Summary of the Process**

1. **The ODE system** describes the motion of the ball in both the horizontal and vertical directions, with the velocities and positions evolving over time.
2. **The RK4 method** discretizes these continuous equations into small steps of size $ \Delta t $. For each step, the

 Runge-Kutta method computes intermediate derivatives and combines them to provide an accurate approximation of the solution at the next time step.
3. **The results** provide the trajectory of the ball, showing how its position and velocity change over time, under the influence of gravity.

This process gives a numerically computed trajectory of the ball thrown at an initial velocity, providing insight into its motion and behavior as time progresses.

The goal of this program is to simulate the trajectory of a ball thrown with an initial velocity, using the Runge-Kutta 4th order (RK4) method to numerically integrate the equations of motion.

---

### 1. **Main Program (`ball_throw`)**

This is the entry point of the program. Here's the breakdown:

```fortran
program ball_throw
    use, intrinsic :: iso_fortran_env, only : DP => REAL64, INT32
    use :: rk4_mod, only  : rk4vec
    implicit none
    integer, parameter :: NR_EQS = 4
    integer, parameter :: MAX_STEPS = 10000
    real(kind=DP), parameter :: DELTA_T = 0.01_DP
    real(kind=DP) :: t
    real(kind=DP), dimension(NR_EQS) :: u, u_new
    integer :: step
```

#### Key Declarations:
- **`iso_fortran_env`**: This module is used to access standard data types. Here, `DP` (double precision) is defined as `REAL64` and `INT32` is defined as a 32-bit integer. These are important for ensuring that we use appropriate precision in the calculations.
  
- **`rk4_mod`**: The program uses this module to import the `rk4vec` subroutine, which performs a Runge-Kutta step for a vector of ODEs (our system of motion equations).

- **Parameters**:
  - `NR_EQS = 4`: This is the number of equations in the system (corresponding to the state variables: position and velocity in both x and y directions).
  - `MAX_STEPS = 10000`: The maximum number of iterations (steps) the simulation will run.
  - `DELTA_T = 0.01_DP`: The time step size used for each RK4 integration step.

- **State Vector (`u`)**:
  - `u` is an array of size `NR_EQS = 4`, where each element corresponds to the state of the ball:
    - `X = u(1)`: Position in the x-direction.
    - `Y = u(2)`: Position in the y-direction.
    - `VX = u(3)`: Velocity in the x-direction.
    - `VY = u(4)`: Velocity in the y-direction.

- **`u_new`**: This array will store the updated state vector after each Runge-Kutta step.

- **`step`**: An integer to keep track of the iteration or time step.

#### Initial Conditions:
```fortran
    step = 0
    t = 0.0_DP
    X = 0.0_DP
    Y = 0.0_DP
    VX = 5.0_DP
    VY = 5.0_DP
    print *, step, t, X, Y, VX, VY
```

- The initial time `t` is set to 0.
- The ball is initialized at the origin (`X = 0.0`, `Y = 0.0`).
- The initial velocity in both x and y directions is set to 5.0 m/s.

#### Simulation Loop:
```fortran
    do step = 1, MAX_STEPS
        call rk4vec(t, NR_EQS, u, DELTA_T, rhs, u_new)
        t = t + DELTA_T
        u = u_new
        if (Y < 0.0_DP) exit
        print *, step, t, X, Y, VX, VY
    end do
```

- The `do` loop will run for `MAX_STEPS` iterations (or until `Y < 0.0_DP`, i.e., the ball hits the ground).
- The `rk4vec` subroutine is called to perform a Runge-Kutta step to update the ball's state. This updates the vector `u_new` using the function `rhs` (right-hand side).
- After the integration step, the time `t` is incremented by `DELTA_T`, and the state vector `u` is updated with the new values in `u_new`.
- The program will stop when `Y` (the vertical position) is less than 0 (the ball hits the ground).
- At each step, the program prints the current step number, time `t`, and the state of the ball (`X`, `Y`, `VX`, `VY`).

---

### 2. **Subroutine `rhs` (Right-Hand Side Function)**

The `rhs` subroutine calculates the derivatives of the state variables, which are used in the Runge-Kutta method.

```fortran
subroutine rhs(t, n, u, u_prime)
    implicit none
    real(kind=DP), intent(in) :: t
    integer(kind=INT32) :: n
    real(kind=DP), dimension(n), intent(in) :: u
    real(kind=DP), dimension(n), intent(out) :: u_prime

    X_prime = VX
    Y_prime = VY
    VX_prime = 0.0_DP
    VY_prime = -9.81_DP
end subroutine rhs
```

- **Inputs**:
  - `t`: The current time (though it's not used directly in this subroutine).
  - `n`: The number of state variables (in this case, `n = 4`).
  - `u`: The current state vector (position and velocity).

- **Outputs**:
  - `u_prime`: The derivatives of the state variables (position and velocity).

#### Derivatives:
- `X_prime = VX`: The derivative of the position in the x-direction is the velocity in the x-direction.
- `Y_prime = VY`: The derivative of the position in the y-direction is the velocity in the y-direction.
- `VX_prime = 0`: The acceleration in the x-direction is assumed to be zero (no forces acting in the horizontal direction).
- `VY_prime = -9.81_DP`: The acceleration in the y-direction is due to gravity, which is constant at `-9.81 m/s^2` (negative because it points downward).

These derivatives are used to update the state vector in the Runge-Kutta method.

---

### 3. **Runge-Kutta 4th Order Method (RK4)**

There are two subroutines: `rk4` and `rk4vec`. We will focus on `rk4vec`, as it is used in the main program for vector systems of equations.

#### `rk4vec` (Runge-Kutta for Vector ODEs):

```fortran
subroutine rk4vec(t0, m, u0, dt, f, u)
    implicit none
    integer(kind=4) m
    real(kind=8) dt
    external f
    real(kind=8) f0(m), f1(m), f2(m), f3(m)
    real(kind=8) t0, t1, t2, t3
    real(kind=8) u(m), u0(m), u1(m), u2(m), u3(m)

    call f(t0, m, u0, f0)
    t1 = t0 + dt / 2.0D+00
    u1(1:m) = u0(1:m) + dt * f0(1:m) / 2.0D+00
    call f(t1, m, u1, f1)

    t2 = t0 + dt / 2.0D+00
    u2(1:m) = u0(1:m) + dt * f1(1:m) / 2.0D+00
    call f(t2, m, u2, f2)

    t3 = t0 + dt
    u3(1:m) = u0(1:m) + dt * f2(1:m)
    call f(t3, m, u3, f3)

    u(1:m) = u0(1:m) + (dt / 6.0D+00) * (f0(1:m) + 2.0D+00 * f1(1:m) + 2.0D+00 * f2(1:m) + f3(1:m))
end subroutine rk4vec
```

#### Parameters:
- **`t0`**: The current time.
- **`m`**: The dimension of the system (number of state variables, 4 in this case).
- **`u0`**: The current state vector.
- **`dt`**: The time step.
- **`f`**: A function (subroutine) that computes the derivatives (i.e., the `rhs` function).
- **`u`**: The updated state vector after one RK4 step.

#### Process:
1. **First Step**: `call f(t0, u0, f0)` calculates the derivative at the current state. Then the time is incremented by `dt/2`, and the state is updated with the first approximation `u1`.
2. **Second Step**: The process repeats for `u1`, `u2`, and `u3`, each time computing the derivative at the midpoint or full step.
3. **Final Update**: The four approximations (stored in `f0`, `f1`, `f2`, `f3`) are combined using a weighted average to give the new state

 `u`.

This method provides a high degree of accuracy for solving ODEs.

---

### 4. **Timestamp Subroutine**

```fortran
subroutine timestamp()
    ...
end subroutine timestamp
```

This subroutine prints the current date and time in a human-readable format. While not used directly in the ball-throw simulation, it can be useful for logging or debugging purposes.

---

### Summary of Key Components:
- **Main Program**: Sets up initial conditions and simulates the ball throw using RK4 integration.
- **RK4 Method**: Used to numerically solve the system of differential equations governing the motion of the ball.
- **RHS (Right-Hand Side)**: Computes the derivatives of the system's state variables (position and velocity).
- **RK4 Integration**: Updates the system’s state by applying the 4th-order Runge-Kutta method.

This program simulates the motion of a ball under the influence of gravity using numerical methods. If you'd like to further improve or modify any parts of the code (e.g., adding air resistance or changing initial conditions), I can guide you through those changes as well.

## Program Code

section_ball_throw.f90

```{literalinclude} Fortran_Code/Section_Ball_Throw/app/section_ball_throw.f90
---
language: fortran
---
```

rk4.f90

```{literalinclude} Fortran_Code/Section_Ball_Throw/src/rk4.f90
---
language: fortran
---
```

The above program is compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [3]:
import os
root_dir = ""
root_dir = os.getcwd()

The following FPM configuration file (fpm.toml) was used:

```{literalinclude} Fortran_Code/Section_Ball_Throw/fpm.toml
---
language: toml
---
```

In [4]:
code_dir = root_dir + "/" + "Fortran_Code/Section_Ball_Throw"

In [5]:
os.chdir(code_dir)

In [6]:
build_status = os.system("fpm build 2>/dev/null")

 + mkdir -p build/dependencies
[  0%]                        rk4.f90
[ 25%]                        rk4.f90  done.

././src/rk4.f90:158:26:

  158 |   call f ( t0, m, u0, f0 )
      |                          1
Error: Procedure 'f' called with an implicit interface at (1) [-Werror=implicit-interface]
compilation terminated due to -fmax-errors=1.


## Run the Program using FPM (Fortran Package Manager)

In [7]:
exec_status = \
    os.system("fpm run")

           1           2           3           4           5


Project is up to date
At line 11 of file app/section_arrays_double_deallocate.f90
Fortran runtime error: Attempt to DEALLOCATE unallocated 'values'

Error termination. Backtrace:
#0  0x7f536dc218c2 in ???
#1  0x7f536dc223b9 in ???
#2  0x7f536dc22949 in ???
#3  0x561b9535074a in double_deallocate
	at app/section_arrays_double_deallocate.f90:11
#4  0x561b953507a4 in main
	at app/section_arrays_double_deallocate.f90:12
<ERROR> Execution for object " Section_Arrays_Double_Deallocate " returned exit code  2
<ERROR> *cmd_run*:stopping due to failed executions
STOP 2
